_______________________________

With distance model
_______________________________

In [2]:
import gymnasium as gym
import numpy as np
import torch
import os

import sys
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.env_wrappers import EnvBuilder
from src.ppo_agent_v2 import PPOAgent
from src.env_wrappers import GoalObservationWrapper
import src.distance_models as distance_models
from src.utils import trajectories_to_dataset

os.environ['WANDB_API_KEY'] = 'd58c31e07030724bbab6b7e8edc93edacd934c87'

# Check for GPU
print("CUDA available:", torch.cuda.is_available())
print("Current device:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(torch.cuda.current_device()) if torch.cuda.is_available() else "CPU")

CUDA available: True
Current device: 0
Device name: Tesla V100-SXM3-32GB


In [3]:
env_id = 'PointMaze_UMaze-v3'
train_episodes = 10000
max_episode_steps = 512
seed = 0

torch.manual_seed(seed); np.random.seed(seed)

# Initialize environment
c = 'c'; U_map = [
    [1, 1, 1, 1, 1],
    [1, c, 0, 0, 1],
    [1, 1, 1, 0, 1],
    [1, c, 0, 0, 1],
    [1, 1, 1, 1, 1]
]
builder = EnvBuilder(env_id=env_id, maze_map=U_map, max_episode_steps=max_episode_steps, seed=seed)
obs_dim, act_dim = builder.get_obs_act_dim()

In [4]:
N_envs = 1
def make_env(env_id, idx, capture_video, run_name, gamma):
    def thunk():
        # Set render_mode to 'rgb_array' if capturing video
        render_mode = "rgb_array" if capture_video and idx == 0 else None
        env = gym.make(env_id, render_mode=render_mode)
        # env = gym.wrappers.FlattenObservation(env)
        env = GoalObservationWrapper(env)
        if capture_video and idx == 0:
            env = gym.wrappers.RecordVideo(env, f"videos/{run_name}")
        env = gym.wrappers.RecordEpisodeStatistics(env)
        return env
    return thunk

In [ ]:
def train_supervised_distance(
        envs,
        env,
        agent,
        distance_model,
        *,
        num_stage: int = 30,
        eval_size: int = 10,
        max_episode_steps: int = 350,
        d_samples: int = 500000,
        en_reward=False,
    ):
        sup_model = distance_model
        for episode in range(num_stage):
            agent.train_ppo(envs, use_distance_shaping = en_reward, verbose=False)
            trajectories = agent.evaluate_ppo(env, num_episodes=eval_size, max_episode_steps=max_episode_steps)

            dataset = trajectories_to_dataset(trajectories, samples=d_samples)
            # sup_model = SupervisedDistanceEstimator(input_dim=4)
            sup_loss = sup_model.train_from_data(dataset, epochs=20, batch_size=16192)          
            print(f"Finish {episode=}, dd loss = {sup_loss}\n")


In [7]:
envs = gym.vector.SyncVectorEnv([make_env(env_id, seed, 0, False, "test")])
env = builder()
distance_model = distance_models.SupervisedDistanceEstimator(input_dim=4)

In [9]:
EXPERIMENTS = {
    # tag      include_distance_state  use_distance_shaping
    "base"            : (False, False),
    "reward_only"     : (False, True ),
    "state_only"      : (True , False),
    "reward_and_state": (True , True ),
}


In [ ]:
en_state, en_reward = EXPERIMENTS["state_only"]

agent = PPOAgent(state_dim=obs_dim, action_dim=act_dim, total_timesteps=10000, distance_model=distance_model, include_distance_state=en_state)
agent.train_ppo(envs, use_distance_shaping = True)
train_supervised_distance(envs, env, agent, distance_model, en_reward=en_reward)

wandb: Currently logged in as: gamershmidt-sofya (gamershmidt-sofya-innopolis-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
/home/user7/.venv/lib/python3.10/site-packages/wandb/analytics/sentry.py:258: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}
/home/user7/.venv/lib/python3.10/site-packages/wandb/analytics/sentry.py:258: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}


global_step=300, episodic_return=[0.]
global_step=600, episodic_return=[160.]
global_step=900, episodic_return=[0.]
global_step=1200, episodic_return=[0.]
global_step=1500, episodic_return=[204.]
global_step=1800, episodic_return=[51.]
global_step=2100, episodic_return=[182.]
global_step=2400, episodic_return=[0.]
global_step=2700, episodic_return=[0.]
global_step=3000, episodic_return=[0.]
global_step=3300, episodic_return=[0.]
global_step=3600, episodic_return=[0.]
global_step=3900, episodic_return=[0.]
global_step=4200, episodic_return=[0.]
global_step=4500, episodic_return=[0.]
global_step=4800, episodic_return=[0.]
global_step=5100, episodic_return=[0.]
global_step=5400, episodic_return=[0.]
global_step=5700, episodic_return=[0.]
global_step=6000, episodic_return=[0.]
global_step=6300, episodic_return=[0.]
global_step=6600, episodic_return=[101.]
global_step=6900, episodic_return=[0.]
global_step=7200, episodic_return=[0.]
global_step=7500, episodic_return=[0.]
global_step=7800, e

Evaluating: 100%|███████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.05it/s]


Success rate: 0.00%, avg_return: 0.00


Building dataset: 100%|████████████████████████████████████████████| 10/10 [00:00<00:00, 274.46it/s]


...

Success rate: 40.00%, avg_return: 164.25


Building dataset: 100%|████████████████████████████████████████████| 10/10 [00:00<00:00, 368.16it/s]


Trajectories processed : 10
Generated samples      : 341272 (train 273017 / eval 68255)


Training, t_loss=0.00: 100%|████████████████████████████████████████| 20/20 [00:00<00:00, 26.93it/s]


eval_losses[0]=0.001771823457462623 -> eval_losses[-1]=0.0004764316702192934
Finish episode=26, dd loss = 0.00044820268522016704


In [ ]:
agent.save_model()

In [ ]:
distance_model.plot_distance_heatmap(env, source_point=(-1, 1))

In [ ]:
model_path = "results/ppo_r/PPO_R.cleanrl_model"
agent.agent.load_state_dict(torch.load(model_path, map_location="cuda"))
_ = evaluate_ppo(agent, env, num_episodes=100, max_episode_steps=300)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import wandb
import numpy as np
import mujoco 

from tqdm import trange

def plot_distance_heatmap(model, env=None, source_point=None, xlim=None, ylim=None, grid_size=500, cmap="viridis", wall_alpha=0.35, epsilon=0.1):
    """Heat-map of model-predicted distances with MuJoCo walls overlaid."""
    source_point = model.start_point if source_point is None else source_point
    xlim = model.xlim if xlim is None else xlim
    ylim = model.ylim if ylim is None else ylim
    x_src, y_src = source_point

    # ── 1. build query grid ────────────────────────────────────────────────
    xs = np.linspace(*xlim, grid_size)
    ys = np.linspace(*ylim, grid_size)
    xx, yy = np.meshgrid(xs, ys)
    targets = np.stack([xx.ravel(), yy.ravel()], 1)
    inp = np.hstack([np.full((targets.shape[0], 2), [x_src, y_src]), targets])

    # ── 2. model prediction ───────────────────────────────────────────────
    with torch.no_grad():
        pred = model(torch.as_tensor(inp, device=model.device, dtype=torch.float32)).squeeze().cpu().numpy()
    heatmap = pred.reshape(grid_size, grid_size)

    # ── 3. plot heat-map ──────────────────────────────────────────────────
    fig, ax = plt.subplots(figsize=(7, 6))
    im = ax.imshow(heatmap, origin="lower", extent=(*xlim, *ylim),
                cmap=cmap, aspect="equal")
    fig.colorbar(im, ax=ax, label="Predicted distance")

    # ── 4. overlay walls ──────────────────────────────────────────────────
    if env:
        mjm = env.unwrapped.model

        def geom_name(i):
            if hasattr(mjm, "geom_names"):
                raw = mjm.geom_names[i]
                return raw.decode() if isinstance(raw, (bytes, bytearray)) else raw
            return mujoco.mj_id2name(mjm, mujoco.mjtObj.mjOBJ_GEOM, i) or ""

        for gid in range(mjm.ngeom):
            name = geom_name(gid).lower()
            if "block" in name:
                cx, cy = mjm.geom_pos[gid, :2]
                hx, hy = mjm.geom_size[gid, :2]
                ax.add_patch(patches.Rectangle(
                    (cx - hx, cy - hy), 2 * hx, 2 * hy,
                    facecolor="black", alpha=wall_alpha, edgecolor="none"))

    # ── 5. decorations ────────────────────────────────────────────────────
    ax.scatter(*source_point, color="red", s=60, edgecolors="black", label="Source")
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_title("Distance heat-map with maze")
    ax.grid()
    ax.legend()

    # ── 6. axis limits and equal scaling ──────────────────────────────────
    xeps = epsilon * (xlim[1] - xlim[0])
    yeps = epsilon * (ylim[1] - ylim[0])
    ax.set_xlim(xlim[0] - xeps, xlim[1] + xeps)
    ax.set_ylim(ylim[0] - yeps, ylim[1] + yeps)
    ax.set_aspect("equal")

    plt.tight_layout()
    plt.show()
plot_distance_heatmap(distance_model, env, source_point=(-1, -1))

In [ ]:
trajectories = agent.evaluate_ppo(env, num_episodes=100, max_episode_steps=350)

In [ ]:
def evaluate_ppo(
    agent,
    env,
    *,
    num_episodes: int = 100,
    max_episode_steps: int | None = 170,
):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    agent.agent.to(device)         
    agent.agent.eval()

    trajectories = []
    success_count = 0
    avg_return = 0

    for ep in trange(num_episodes, desc="Evaluating", ncols=100):
        obs, _ = env.reset()
        episode_traj = []

        for step in range(max_episode_steps):
            ball_x, ball_y = obs[:2]
            episode_traj.append((ball_x, ball_y))

            obs_tensor = torch.tensor(obs, dtype=torch.float32, device=device).unsqueeze(0)
            with torch.no_grad():
                action, *_ = agent.agent.get_action_and_value(obs_tensor)

            obs, _, terminated, truncated, info = env.step(action.squeeze(0).cpu().numpy())

            if info.get("success", False):
                success_count += 1
                avg_return += max_episode_steps - step
                break
            if terminated or truncated:
                break

        trajectories.append(episode_traj)

    print(f"Success rate over {num_episodes} episodes: {success_count / num_episodes:.2%}, avg_return = {avg_return / success_count if success_count else 0 :.2f}")
    return trajectories


In [ ]:
print(f"Success rate over {num_episodes} episodes: {success_count / num_episodes:.2%}, {avg_return =}")
evaluate_ppo(agent, env, num_episodes=1, max_episode_steps=300)